In [4]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
import torch
import torch.utils.data
# from torchvision import datasets, transforms
import numpy as np
import argparse
from udlp.autoencoder.denoisingAutoencoder import DenoisingAutoencoder
from pacdataset import PacDataset

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
batch_size = 4
lr = 0.001
epochs = 2

# DEA with PAC

In [6]:
datasets = {x: PacDataset(train=(x=='train'), 
                          root_dir = "/home/paperspace/data/pac2018/") 
            for x in ['val','train']}

train_loader = torch.utils.data.DataLoader(
    datasets['train'],
    batch_size= batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    datasets['val'],
    batch_size= batch_size, shuffle=False, num_workers=2)

In [30]:
in_features = 17545
out_features = 500
dae = DenoisingAutoencoder(in_features, out_features)
dae.fit(train_loader, test_loader, lr=lr, num_epochs=epochs, loss_type="cross-entropy")

=====Denoising Autoencoding layer=======
#Epoch 0: Valid Reconstruct Loss: 12171.869
#Epoch   1: Reconstruct Loss: 3515.545, Valid Reconstruct Loss: 3414.069
#Epoch   2: Reconstruct Loss: 3339.725, Valid Reconstruct Loss: 3278.801


# SDEA with PAC

In [7]:
from udlp.autoencoder.stackedDAE import StackedDAE


In [ ]:
# in_features = 784
in_features = 17545
out_features = 500
pretrainepochs = 10 

sdae = StackedDAE(input_dim=in_features, z_dim=10, binary=True,
    encodeLayer=[500,500,2000], decodeLayer=[2000,500,500], activation="relu", 
    dropout=0)
sdae.pretrain(train_loader, test_loader, lr=lr, batch_size= batch_size, 
    num_epochs=pretrainepochs, corrupt=0.3, loss_type="cross-entropy")
sdae.save_model("model/sdae.pt")
sdae.fit(train_loader, test_loader, lr= lr, num_epochs= epochs, corrupt=0.3, loss_type="cross-entropy")


l is  1
=====Denoising Autoencoding layer=======
#Epoch 0: Valid Reconstruct Loss: 12173.703
#Epoch   1: Reconstruct Loss: 3448.276, Valid Reconstruct Loss: 3403.032
#Epoch   2: Reconstruct Loss: 3294.175, Valid Reconstruct Loss: 3389.241
#Epoch   3: Reconstruct Loss: 3252.883, Valid Reconstruct Loss: 3308.697
#Epoch   4: Reconstruct Loss: 3209.897, Valid Reconstruct Loss: 3323.170
#Epoch   5: Reconstruct Loss: 3178.561, Valid Reconstruct Loss: 3306.884
#Epoch   6: Reconstruct Loss: 3158.802, Valid Reconstruct Loss: 3252.964
#Epoch   7: Reconstruct Loss: 3143.454, Valid Reconstruct Loss: 3263.911
#Epoch   8: Reconstruct Loss: 3128.448, Valid Reconstruct Loss: 3233.133
#Epoch   9: Reconstruct Loss: 3116.606, Valid Reconstruct Loss: 3284.641
